In [1]:
import json

def extraer_categorias_y_generos(json_file):
    # Abrimos y cargamos el archivo JSON
    with open(json_file, 'r', encoding='utf-8') as file:
        datos = json.load(file)

    # Conjuntos para almacenar categorías y géneros únicos
    categorias = set()
    generos = set()

    # Procesamos cada objeto en el archivo JSON
    for entrada in datos:
        categorias.add(entrada['categoria'])  # Extraemos la categoría
        generos.update(entrada['genero'].split(', '))  # Dividimos los géneros por coma y espacio

    # Devolvemos listas de categorías y géneros únicos
    return list(categorias), list(generos)

def generar_inserts_categorias_generos(categorias, generos):
    consultas_categoria = []
    consultas_genero = []

    # Generamos las consultas INSERT para la tabla Categoria
    for idx, categoria in enumerate(categorias, start=1):
        categoria = categoria.replace("'", "''")  # Escapamos comillas
        consultas_categoria.append(f"INSERT INTO Categoria (id, nombre) VALUES ({idx}, '{categoria}');")

    # Generamos las consultas INSERT para la tabla Genero
    for idx, genero in enumerate(generos, start=1):
        genero = genero.replace("'", "''")  # Escapamos comillas
        #print(f'"{genero}":{idx},')
        consultas_genero.append(f"INSERT INTO Genero (id, nombre) VALUES ({idx}, '{genero}');")
    # Devolvemos ambas listas de consultas
    return consultas_categoria, consultas_genero

# Uso del script
json_file = 'trailerflix.json'  # Nombre del archivo JSON

# Extraemos las categorías y géneros únicos
categorias, generos = extraer_categorias_y_generos(json_file)

# Generamos las consultas INSERT para categorías y géneros
consultas_categoria, consultas_genero = generar_inserts_categorias_generos(categorias, generos)

# Imprimimos las consultas generadas para las tablas Categoria y Genero
print("Consultas para la tabla Categoria:")
for consulta in consultas_categoria:
    print(consulta)

print("\nConsultas para la tabla Genero:")
for consulta in consultas_genero:
    print(consulta)


Consultas para la tabla Categoria:
INSERT INTO Categoria (id, nombre) VALUES (1, 'Serie');
INSERT INTO Categoria (id, nombre) VALUES (2, 'Película');

Consultas para la tabla Genero:
INSERT INTO Genero (id, nombre) VALUES (1, 'Comedia');
INSERT INTO Genero (id, nombre) VALUES (2, 'Acción');
INSERT INTO Genero (id, nombre) VALUES (3, 'Ciencia Ficción');
INSERT INTO Genero (id, nombre) VALUES (4, 'Fantasía');
INSERT INTO Genero (id, nombre) VALUES (5, 'Western');
INSERT INTO Genero (id, nombre) VALUES (6, 'Futurista');
INSERT INTO Genero (id, nombre) VALUES (7, '¿Ficción?');
INSERT INTO Genero (id, nombre) VALUES (8, 'Drama');
INSERT INTO Genero (id, nombre) VALUES (9, 'Crimen');
INSERT INTO Genero (id, nombre) VALUES (10, 'Suspenso');
INSERT INTO Genero (id, nombre) VALUES (11, 'Animación');
INSERT INTO Genero (id, nombre) VALUES (12, 'Historia');
INSERT INTO Genero (id, nombre) VALUES (13, 'Aventura');
INSERT INTO Genero (id, nombre) VALUES (14, 'Suceso Real');
INSERT INTO Genero (id, 

In [8]:

# Mapeos de categoría y género a sus respectivos IDs
categoria_map = {
    "Serie": 1,  # ID de la categoría 'Serie' en la tabla Categoria
    "Película": 2  # Ejemplo de otra categoría
}

genero_map = {
    "Música":1,
    "Familia":2,
    "Romance":3,
    "Drama":4,
    "Sci-Fi":5,
    "Sucesos":6,
    "Crimen":7,
    "Tecnología":8,
    "Suceso Real":9,
    "Acción":10,
    "Ficción":11,
    "Terror":12,
    "¿Ficción?":13,
    "Ciencia Ficción":14,
    "Suspenso":15,
    "Fantasía":16,
    "Aventura":17,
    "Comedia":18,
    "Historia":19,
    "Ciencia ficción":20,
    "Intriga":21,
    "Musical":22,
    "Misterio":23,
    "Animación":24,
    "Western":25,
    "Futurista":26
}

def generar_inserts_json(json_file, table_name):
    # Abrimos y cargamos el archivo JSON
    with open(json_file, 'r', encoding='utf-8') as file:
        datos = json.load(file)

    # Excluimos 'categoria' y 'genero', e incluimos 'categoria_id'
    columnas_excluidas = {'categoria', 'genero', 'gen','reparto'}
    columnas = [col for col in datos[0].keys() if col not in columnas_excluidas]
    columnas.append('categoria_id')  # Añadimos la clave foránea de categoría

    # Lista para almacenar las consultas INSERT generadas para contenido
    consultas_insert_contenido = []

    # Procesamos cada objeto en el archivo JSON
    for entrada in datos:
        valores = []
        # Iteramos sobre las columnas para construir los valores
        for col in columnas:
            if col == 'categoria_id':
                # Usamos el mapeo para obtener el categoria_id
                categoria_id = categoria_map.get(entrada['categoria'], 'NULL')
                valores.append(str(categoria_id))
            else:
                valor = entrada[col]
                if isinstance(valor, str):
                    # Escapamos las comillas simples en los valores de tipo string
                    valor = valor.replace("'", "''")
                    valores.append(f"'{valor}'")
                else:
                    valores.append(str(valor))

        # Creamos la consulta INSERT para la tabla contenido
        insert_query_contenido = f"INSERT INTO {table_name} ({', '.join(columnas)}) VALUES ({', '.join(valores)});"

        # Añadimos la consulta a la lista
        consultas_insert_contenido.append(insert_query_contenido)

    # Generamos las consultas INSERT para la tabla de unión contenido_genero
    consultas_insert_genero = []
    for entrada in datos:
        # Obtenemos el contenido_id (puede que sea el último insertado o a partir del JSON si tiene un id)
        contenido_id = entrada['id']  # O usa el id autoincremental si lo has manejado en otro lado
        genero_ids = [genero_map.get(g.strip(), 'NULL') for g in entrada['genero'].split(', ')]

        for genero_id in genero_ids:
            if genero_id != 'NULL':
                consultas_insert_genero.append(f"INSERT INTO contenido_generos (contenido_id, genero_id) VALUES ({contenido_id}, {genero_id});")

    # Devolvemos ambas listas de consultas
    return consultas_insert_contenido, consultas_insert_genero


In [ ]:
import json

# Función para generar consultas INSERT para actores y contenido_actores
def generar_inserts_actores(json_file):
    # Abrimos y cargamos el archivo JSON
    with open(json_file, 'r', encoding='utf-8') as file:
        datos = json.load(file)

    # Diccionario para almacenar los actores únicos
    actores_unicos = {}
    # Lista para almacenar las consultas INSERT para actores
    consultas_insert_actores = []
    # Lista para almacenar las consultas INSERT para la tabla intermedia contenido_actores
    consultas_insert_contenido_actores = []
    
    actor_id_counter = 1  # Contador manual para asignar ID a los actores (suponiendo que no existe en la BD)
    
    # Procesamos cada entrada en el JSON
    for entrada in datos:
        contenido_id = entrada['id']
        actores = [actor.strip() for actor in entrada['reparto'].split(',')]
        
        for actor in actores:
            if actor not in actores_unicos:
                # Asignamos un nuevo ID al actor y lo guardamos en actores_unicos
                actores_unicos[actor] = actor_id_counter
                # Escapamos las comillas simples correctamente en los nombres de los actores
                actor_escaped = actor.replace("'", "''")
                insert_query_actor = f"INSERT INTO actores (id, nombre) VALUES ({actor_id_counter}, '{actor_escaped}');"
                consultas_insert_actores.append(insert_query_actor)
                actor_id_counter += 1
            
            # Creamos la consulta para insertar la relación en contenido_actores
            actor_id = actores_unicos[actor]
            insert_query_contenido_actores = f"INSERT INTO contenido_actores (contenido_id, actor_id) VALUES ({contenido_id}, {actor_id});"
            consultas_insert_contenido_actores.append(insert_query_contenido_actores)
    
    # Devolvemos todas las consultas generadas
    return consultas_insert_actores, consultas_insert_contenido_actores

# Uso del script
json_file = 'trailerflix.json'  # Nombre del archivo JSON

# Generamos las consultas INSERT
consultas_actores, consultas_contenido_actores = generar_inserts_actores(json_file)

# Imprimimos las consultas generadas para actores
print("Consultas para la tabla actores:")
for consulta in consultas_actores:
    print(consulta)

# Imprimimos las consultas generadas para contenido_actores
print("\nConsultas para la tabla contenido_actores:")
for consulta in consultas_contenido_actores:
    print(consulta)

In [ ]:
# Uso del script
json_file = 'trailerflix.json'  # Nombre del archivo JSON
table_name = 'contenido'         # Nombre de la tabla MySQL

# Generamos las consultas INSERT
consultas_contenido, consultas_genero = generar_inserts_json(json_file, table_name)

# Imprimimos las consultas generadas
print("Consultas para la tabla contenido:")
for consulta in consultas_contenido:
    print(consulta)

print("\nConsultas para la tabla contenido_generos:")
for consulta in consultas_genero:
    print(consulta)